<a href="https://colab.research.google.com/github/m-zayed5722/Miscellaneous-Projects/blob/main/unsupervised_anomaly_swat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
import plotly.graph_objects as go

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
df2 = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/datasets/SWaT_Dataset_Attack_v0.xlsx')

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
attack = df2.copy(deep=True)
attack.columns = attack.iloc[0]
attack = attack.iloc[1: , :]

In [ ]:
target_mapping = {
    'Normal': 1,
    'Attack': -1,
    'A ttack': -1
}
attack["Normal/Attack"] = attack["Normal/Attack"].map(lambda x: target_mapping[x])

In [ ]:
attack['Normal/Attack'].value_counts()

 1    395298
-1     54621
Name: Normal/Attack, dtype: int64

In [ ]:
y = attack['Normal/Attack']
X = attack.iloc[: , :-1]
#dates
X = X.iloc[:,1:]

In [ ]:
x1 = X.copy(deep=True)
x1 = x1.apply(pd.to_numeric)

In [ ]:
#X.head(5)
x1.head(5)

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
1,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,0.307786,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1
2,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,0.307786,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1
3,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,2,...,0.308619,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1
4,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,2,...,0.308619,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1
5,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,2,...,0.308619,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1


In [ ]:
#print summary table
x1 = x1.reindex(x1.var().sort_values().index, axis=1)
inf_df = x1.describe()
inf_df = inf_df.T
inf_df['std'] = inf_df['std']**2
inf_df['median'] = x1.median()
inf_df = inf_df.T
inf_df = inf_df.drop('count')
inf_df = inf_df.drop('25%')
inf_df = inf_df.drop('50%')
inf_df = inf_df.drop('75%')
save_df = inf_df.rename(index={"std": "var"})
save_df = save_df.T
save_df = save_df[['var', 'mean', 'median', 'min', 'max']]
save_df

,var,mean,median,min,max
0,,,,,
P603,0.000000,1.000000,1.000000,1.000000,1.000000
P601,0.000000,1.000000,1.000000,1.000000,1.000000
P502,0.000000,1.000000,1.000000,1.000000,1.000000
P404,0.000000,1.000000,1.000000,1.000000,1.000000
P401,0.000000,1.000000,1.000000,1.000000,1.000000
P301,0.000000,1.000000,1.000000,1.000000,1.000000
P202,0.000000,1.000000,1.000000,1.000000,1.000000
AIT401,0.000015,148.805855,148.808000,148.759900,148.856100
P206,0.000122,1.000122,1.000000,1.000000,2.000000


In [ ]:
#take top 7
top_7 = pd.DataFrame()
top_7['LIT401'] = x1['LIT401']
top_7['LIT101'] = x1['LIT101']
top_7['LIT301'] = x1['LIT301']
top_7['PIT501'] = x1['PIT501']
top_7['PIT503'] = x1['PIT503']
top_7['AIT402'] = x1['AIT402']
top_7[' AIT201'] = x1[' AIT201']


In [ ]:
#duplicate columns: LIT401 , LIT101, LIT301, PIT501, PIT503
x1['LIT401a'] = x1.loc[:, 'LIT401']
x1['LIT401b'] = x1.loc[:, 'LIT401']
x1['LIT401c'] = x1.loc[:, 'LIT401']
x1['LIT401d'] = x1.loc[:, 'LIT401']
x1['LIT401e'] = x1.loc[:, 'LIT401']
x1['LIT401f'] = x1.loc[:, 'LIT401']
x1['LIT401g'] = x1.loc[:, 'LIT401']
x1['LIT401h'] = x1.loc[:, 'LIT401']
x1['LIT401i'] = x1.loc[:, 'LIT401']
x1['LIT401j'] = x1.loc[:, 'LIT401']

In [ ]:
top_7.head(5)

,LIT401,LIT101,LIT301,PIT501,PIT503,AIT402,AIT201
1,942.0662,522.8467,956.1651,250.8652,189.5988,156.0882,262.0161
2,942.0277,522.8860,956.1651,250.8652,189.6789,156.0882,262.0161
3,941.8739,522.8467,956.4855,250.8812,189.6789,156.0882,262.0161
4,941.7970,522.9645,956.8060,250.8812,189.6148,156.0882,262.0161
5,942.2200,523.4748,957.0864,250.8812,189.5027,156.0882,262.0161


In [ ]:
p_df = top_7

In [ ]:
iforest = IsolationForest(contamination=0.1, max_samples='auto', max_features=1.0, bootstrap=False, n_jobs=-1, random_state=1)
pred= iforest.fit_predict(top_7)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
kmean = KMeans(n_clusters=2)
#Kmean.fit(X)
#y_pred = kmean.fit_predict(p_df[['FIT101', 'LIT101', ' MV101', 'P101', 'P102', ' AIT201', 'AIT202', 'AIT203', 'FIT201', ' MV201', ' P201', ' P202', 'P203', ' P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', ' MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501', 'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602', 'P603', 'LIT401a', 'LIT401b', 'LIT401c', 'LIT401d', 'LIT401e', 'LIT401f', 'LIT401g', 'LIT401h', 'LIT401i', 'LIT401j']])
y_pred = kmean.fit_predict(p_df[[ 'LIT401', 'LIT101', 'LIT301', 'PIT501', 'PIT503',  'AIT402',' AIT201' ]])

#principalDf['Cluster'] = y_pred

In [ ]:
plot_df = pd.DataFrame()
plot_df['datetime'] = attack[' Timestamp']
plot_df['Acutal_Attack/Normal'] = attack['Normal/Attack']
plot_df['Pred_Clustering'] = y_pred
plot_df['Pred_IF'] = pred

In [ ]:
target_mapping = {
    0: 1,
    1: -1,
}
plot_df['Pred_Clustering'] = plot_df['Pred_Clustering'].map(lambda x: target_mapping[x])
plot_df['Pred_Clustering'].value_counts()

-1    413095
 1     36824
Name: Pred_Clustering, dtype: int64

In [ ]:
plot_df['Pred_IF'].value_counts()

 1    404928
-1     44991
Name: Pred_IF, dtype: int64

In [ ]:
report = classification_report(y, plot_df['Pred_Clustering'])
print(report)

              precision    recall  f1-score   support

          -1       0.89      0.60      0.72     54621
           1       0.95      0.99      0.97    395298

    accuracy                           0.94    449919
   macro avg       0.92      0.80      0.84    449919
weighted avg       0.94      0.94      0.94    449919



In [ ]:
report = classification_report(y, plot_df['Pred_IF'])
print(report)

              precision    recall  f1-score   support

          -1       0.57      0.70      0.63     54621
           1       0.96      0.93      0.94    395298

    accuracy                           0.90    449919
   macro avg       0.76      0.81      0.78    449919
weighted avg       0.91      0.90      0.90    449919



In [ ]:
#duplicate col =1
report = classification_report(y, plot_df['Pred_Clustering'])
print(report)

              precision    recall  f1-score   support

          -1       0.05      0.40      0.09     54621
           1       0.11      0.01      0.02    395298

    accuracy                           0.06    449919
   macro avg       0.08      0.20      0.06    449919
weighted avg       0.10      0.06      0.03    449919



In [ ]:
#duplicate col =1
report = classification_report(y, plot_df['Pred_IF'])
print(report)

              precision    recall  f1-score   support

          -1       0.61      0.76      0.68     54621
           1       0.97      0.93      0.95    395298

    accuracy                           0.91    449919
   macro avg       0.79      0.85      0.81    449919
weighted avg       0.92      0.91      0.92    449919



In [ ]:
#10%
report = classification_report(y, plot_df['Pred_IF'])
print(report)

              precision    recall  f1-score   support

          -1       0.80      0.66      0.73     54621
           1       0.95      0.98      0.97    395298

    accuracy                           0.94    449919
   macro avg       0.88      0.82      0.85    449919
weighted avg       0.94      0.94      0.94    449919



In [ ]:
target_mapping = {
    1: 0,
    -1: -1,
}
plot_df['Acutal_Attack/Normal'] = plot_df['Acutal_Attack/Normal'].map(lambda x: target_mapping[x])
plot_df['Acutal_Attack/Normal'].value_counts()

 0    395298
-1     54621
Name: Acutal_Attack/Normal, dtype: int64

In [ ]:
target_mapping = {
    1: 0.5,
    0: 1.5,
}
plot_df['Pred_Clustering'] = plot_df['Pred_Clustering'].map(lambda x: target_mapping[x])
plot_df['Pred_Clustering'].value_counts()

1.5    413169
0.5     36750
Name: Pred_Clustering, dtype: int64

In [ ]:
target_mapping = {
    1: 2,
    -1: 3,
}
plot_df['Pred_IF'] = plot_df['Pred_IF'].map(lambda x: target_mapping[x])
plot_df['Pred_IF'].value_counts()

2    382431
3     67488
Name: Pred_IF, dtype: int64

In [ ]:
target_mapping = {
    0: 'Normal',
    -1: 'Attack',
}
plot_df['Acutal_Attack/Normal'] = plot_df['Acutal_Attack/Normal'].map(lambda x: target_mapping[x])
plot_df['Acutal_Attack/Normal'].value_counts()

Normal    395298
Attack     54621
Name: Acutal_Attack/Normal, dtype: int64

In [ ]:
target_mapping = {
    1.5: 'Normal2',
    0.5: 'Attack2',
}
plot_df['Pred_Clustering'] = plot_df['Pred_Clustering'].map(lambda x: target_mapping[x])
plot_df['Pred_Clustering'].value_counts()

Normal2    413169
Attack2     36750
Name: Pred_Clustering, dtype: int64

In [ ]:
target_mapping = {
    2: 'Normal3',
    3: 'Attack3',
}
plot_df['Pred_IF'] = plot_df['Pred_IF'].map(lambda x: target_mapping[x])
plot_df['Pred_IF'].value_counts()

Normal3    382431
Attack3     67488
Name: Pred_IF, dtype: int64

Normal    395298
Attack     54621
Name: Acutal_Attack/Normal, dtype: int64

In [ ]:
new_df = plot_df.tail(225000)

In [ ]:
plot_df.shape

(449919, 4)

In [ ]:
fig = px.line(new_df, x='datetime' , y=['Acutal_Attack/Normal', "Pred_Clustering", 'Pred_IF' ])
fig

In [ ]:
plot_df.head(5)

,datetime,Acutal_Attack/Normal,Pred_Clustering,Pred_IF
1,28/12/2015 10:00:00 AM,Normal,Normal2,Normal3
2,28/12/2015 10:00:01 AM,Normal,Normal2,Normal3
3,28/12/2015 10:00:02 AM,Normal,Normal2,Normal3
4,28/12/2015 10:00:03 AM,Normal,Normal2,Normal3
5,28/12/2015 10:00:04 AM,Normal,Normal2,Normal3


In [ ]:
fig = go.Figure()

for column in new_df.columns.to_list():
    fig.add_trace(
        go.Scatter(
            #x = 'datetime',
            x = plot_df.index,
            y = 'datetime',
            #y = new_df[column],
            name = column
        )
    )

fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [False, True, True, True]},
                          {'title': 'All',
                           'showlegend':True}]),
             dict(label = 'Acutal_Attack/Normal',
                  method = 'update',
                  args = [{'visible': [False, True, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Acutal_Attack/Normal',
                           'showlegend':True}]),
             dict(label = 'Pred_Clustering',
                  method = 'update',
                  args = [{'visible': [False, False, True, False]},
                          {'title': 'Pred_Clustering',
                           'showlegend':True}]),
             dict(label = 'Pred_IF',
                  method = 'update',
                  args = [{'visible': [False, False, False, True]},
                          {'title': 'Pred_IF',
                           'showlegend':True}]),
            ])
        )
    ])

fig.show()

ValueError: ignored

In [ ]:
fig = px.line(new_df, x='datetime' , y=["Pred_Cluster",'Acutal_Attack/Normal' ])
fig = fig.add_traces(
    px.scatter(new_df, x='datetime', y='Pred_IF').update_traces(marker_size=6, marker_color="red").data
    )
fig

In [ ]:
#fig = px.scatter(plot_df)
#fig
#fig = px.line(plot_df, x="datetime", y="Predicted_Attack/Normal", color='Acutal_Attack/Normal')
#fig
fig = px.line(new_df, x='datetime' , y="Pred_Cluster")
fig = fig.add_traces(
    px.line(new_df, x='datetime', y='Pred_IF').update_traces(marker_size=6, marker_color="red").data
    )
fig

In [ ]:
report = classification_report(y, p_df['Cluster'])
print(report)

              precision    recall  f1-score   support

          -1       0.89      0.60      0.72     54621
           1       0.95      0.99      0.97    395298

    accuracy                           0.94    449919
   macro avg       0.92      0.80      0.84    449919
weighted avg       0.94      0.94      0.94    449919



In [ ]:
report = classification_report(y, p_df['Cluster'])
print(report)